In [ ]:
def Rainbow_spaghetti_plot_stack(X_data, mode='XES', energyrange=None):
    
    fig, ax = plt.subplots(figsize=(8,15))
    if energyrange is not None: plt.xlim(energyrange)

    
    min_whiteline, max_whiteline = 500, 0
    for compound in X_data:
        x = compound['Normalized']
        whiteline_idx = np.argmax(x)
        if whiteline_idx < min_whiteline:
            min_whiteline = whiteline_idx
        if whiteline_idx > max_whiteline:
            max_whiteline = whiteline_idx
    total_idx_range = max_whiteline - min_whiteline
    
    for compound in X_data:
        x = compound['Normalized']
        whiteline_idx = np.argmax(x)
        scale = (whiteline_idx - min_whiteline)/total_idx_range
        bin_num = int(scale*10)//2
        if bin_num == 5:
            bin_num = 4
        plt.plot(compound['Spectra'][0], compound['Normalized'] + bin_num, '-', color=plt.cm.viridis(scale), alpha=0.1)
                
    plt.title(f"{mode} Spectra", fontsize=30)
    
    plt.xlabel('Energy (eV)', fontsize=26)
    ax.tick_params(direction='in', width=2, length=8)
    plt.xticks(fontsize=20)
    
    if mode == 'XANES' or mode == 'XES':
        ax.xaxis.set_minor_locator(MultipleLocator(2))
        ax.xaxis.set_major_locator(MultipleLocator(10))
    
    ax.tick_params(direction='in', width=2, length=10, which='major')
    ax.tick_params(direction='in', width=1, length=8, which='minor')
    plt.yticks([])

    plt.show()
    
Rainbow_spaghetti_plot_stack(XANES_data, mode='XANES')
Rainbow_spaghetti_plot_stack(XES_data, mode='XES')

In [ ]:
linktopubchem = True

oldlineprops = {}

def highlight_spectra(sel):
    sel.annotation.set_text( \
        str(CIDS[sel.target.index])+','+str(compoundtypes[sel.target.index]))
    if oldlineprops:
        spectra_lines[oldlineprops['index']].set_linewidth(oldlineprops['linewidth'])
        spectra_lines[oldlineprops['index']].set_color(oldlineprops['color'])
        spectra_lines[oldlineprops['index']].set_alpha(oldlineprops['alpha'])
    oldlineprops['index'] = sel.target.index
    oldlineprops['linewidth'] = spectra_lines[sel.target.index].get_lw()
    oldlineprops['color'] = spectra_lines[sel.target.index].get_c()
    oldlineprops['alpha'] = spectra_lines[sel.target.index].get_alpha()
    spectra_lines[sel.target.index].set_linewidth(3)
    spectra_lines[sel.target.index].set_color('red')
    spectra_lines[sel.target.index].set_alpha(1)
    if linktopubchem: webbrowser.open(f"https://pubchem.ncbi.nlm.nih.gov/image/imgsrv.fcgi?cid={CIDS[sel.target.index]}&t=l")

def unhighlight_spectra(sel):
    spectra_lines[sel.target.index].set_linewidth(oldlineprops['linewidth'])
    spectra_lines[sel.target.index].set_color(oldlineprops['color'])
    spectra_lines[sel.target.index].set_alpha(oldlineprops['alpha'])
        

olddotprops = {}

def highlight_dot(sel):
    #sel.annotation.set_text(str(CIDS[sel.target.index])+' '+str(types[sel.target.index]))
    if olddotprops:
        redspace_dots.set_edgecolors(olddotprops['colors'])
    olddotprops['colors'] = copy.deepcopy(redspace_dots.get_edgecolors())
    newdotedgecolors = copy.deepcopy(olddotprops['colors'])
    #webbroswer.open(str(len(olddotedgecolors)))
    #webbrowser.open(repr(sel.artist.get_label().split(',')[0]))
    index = CIDS.index(int(sel.artist.get_label().split(',')[0]))
    newdotedgecolors[index] = [1,0,0,1]
    redspace_dots.set_edgecolors(newdotedgecolors)
    if linktopubchem: webbrowser.open(f"https://pubchem.ncbi.nlm.nih.gov/image/imgsrv.fcgi?cid={CIDS[index]}&t=l")


def unhighlight_dot(sel):
    webbrowser.open('test')
    redspace_dots.set_edgecolors(olddotprops['colors'])
    
spectra_lines = Rainbow_spaghetti_plot_types_stack(plt.subplots(figsize=(8,15)), XANESenergy, XANES, compoundtypes, mode='XANES')
redspace_dots = plot_dim_red(plt.subplots(figsize=(5,5)), 't-SNE', XANES_TSNE, compoundtypes, typecodes, mode='XANES')

#spectra_lines = Rainbow_spaghetti_plot_types_stack(plt.subplots(figsize=(8,15)), XESenergy, XES, compoundtypes, mode='XES')
#redspace_dots = plot_dim_red(plt.subplots(figsize=(5,5)), 't-SNE', XES_TSNE, compoundtypes, typecodes, mode='XES')

dotcursor = mplcursors.cursor(redspace_dots)
dotcursor.connect("add", highlight_spectra)
#dotcursor.connect("remove", unhighlight_spectra)

linecursor = mplcursors.cursor(spectra_lines, highlight=True,
                  highlight_kwargs={'color':'pink', 'alpha':1, 'linewidth':3, 'markeredgewidth':0})
linecursor.connect("add", highlight_dot)
#linecursor.connect("remove", unhighlight_dot)